In [0]:
# =============================================================
# JOB ONLINE — CAPTURA INPE FOCOS (10min)
# =============================================================
# • Processo ONLINE (não é Spark Streaming)
# • Verifica o data server a cada 10 minutos
# • Baixa CSV assim que estiver válido
# • Mantém job ativo por 1 hora
# • Grava arquivos RAW em Volume Databricks
# =============================================================

import re
import time
import requests
from datetime import datetime

# ==============================
# DATA_ALVO
# ==============================
dbutils.widgets.text("DATA_ALVO", "")
DATA_ALVO_RAW = dbutils.widgets.get("DATA_ALVO")

if not DATA_ALVO_RAW:
    DATA_ALVO = datetime.now().strftime("%Y%m%d")
else:
    try:
        DATA_ALVO = datetime.strptime(DATA_ALVO_RAW, "%Y-%m-%d").strftime("%Y%m%d")
    except ValueError:
        DATA_ALVO = DATA_ALVO_RAW

print(f"DATA_ALVO = {DATA_ALVO}")

In [0]:
# ==============================
# CONFIG
# ==============================
BASE_URL = "https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/10min/"
HEADERS = {"User-Agent": "inpe-focos-online/1.0"}
INTERVALO = 600
MIN_TAMANHO = 1024

In [0]:
# ==============================
# PATH VOLUME
# ==============================
VOLUME_BASE = "/Volumes/datamasters/raw/raw_inpe/stream"
LANDING_PATH = f"{VOLUME_BASE}/inpe_in/{DATA_ALVO}"

dbutils.fs.mkdirs(LANDING_PATH)

In [0]:
# ==============================
# LISTA REMOTA
# ==============================
def listar_csvs_remotos():
    r = requests.get(BASE_URL, headers=HEADERS, timeout=30)
    r.raise_for_status()
    arquivos = re.findall(r'"([^"]+\.csv)"', r.text)
    return [a for a in arquivos if DATA_ALVO in a]

In [0]:
# ==============================
# DOWNLOAD (ÚNICO JEITO VÁLIDO)
# ==============================
def baixar_csv(nome):
    url = BASE_URL + nome
    r = requests.get(url, headers=HEADERS, timeout=60)

    if r.status_code != 200:
        return False, "http_error"

    if len(r.content) < MIN_TAMANHO:
        return False, "arquivo_incompleto"

    destino = f"{LANDING_PATH}/{nome}"

    # evita reprocessar
    existentes = [f.name for f in dbutils.fs.ls(LANDING_PATH)]
    if nome in existentes:
        return False, "ja_existe"

    # 🔑 ÚNICA FORMA SEGURA
    dbutils.fs.put(destino, r.text, overwrite=False)

    return True, "ok"

In [0]:
# ==============================
# LOOP ONLINE CONTÍNUO
# ==============================
print("Iniciando captura ONLINE do INPE...")

baixados = set(f.name for f in dbutils.fs.ls(LANDING_PATH))

while True:
    try:
        for nome in listar_csvs_remotos():
            if nome in baixados:
                continue

            sucesso, status = baixar_csv(nome)

            if sucesso:
                print(f"[✔] Baixado {nome}")
                baixados.add(nome)

        time.sleep(INTERVALO)

    except Exception as e:
        print(f"[ERRO] {e}")
        time.sleep(300)